# <font color='blue'>Data Science Academy Big Data Real-Time Analytics com Python e Spark</font>

# <font color='blue'>Capítulo 9</font>

## <font color='blue'>Spark MLLib - Classificação - Decision Tree</font>

<strong> Descrição </strong>
<ul style="list-style-type:square">
  <li>Fácil de compreender e fácil de explicar.</li>
  <li>Variáveis preditoras são usadas para construir uma árvore que progressivamente prevê valores target.</li>
  <li>Dados de treino são usados para construir a árvore de decisão e prever o valor target.</li>
  <li>A árovre de decisão se torna um modelo que é usado para fazer previsões com novos dados.</li>
</ul>

<dl>
  <dt>Vantagens</dt>
  <dd>- Fácil de interpretar e explicar</dd>
  <dd>- Funciona com valores missing</dd>
  <dd>- Veloz</dd>
  <br />
  <dt>Desvantagens</dt>
  <dd>- Acurácia limitada</dd>
  <dd>- Bias podem ocorrer com frequência</dd>
  <dd>- Não funciona bem com muitas variáveis preditoras</dd>
  <br />
  <dt>Aplicação</dt>
  <dd>- Aprovação de crédito</dd>
  <dd>- Categorização preliminar</dd>
</dl>

## Classificar as espécies de flores, listadas no dataset iris

In [1]:
# Credenciais para conexão
# @hidden_cell
credentials_1 = {
  'auth_url':'https://identity.open.softlayer.com',
  'project':'object_storage_f0d6ce32_5e0f_4bc0_8812_229b8d429dbe',
  'project_id':'9a0cc60102244d368e96a83f25d4ca89',
  'region':'dallas',
  'user_id':'0caf8026c98a4342ac027a05416e6dee',
  'domain_id':'3be46074545f4c09b1f10df3ace95998',
  'domain_name':'1351407',
  'username':'member_327b95c3eecf105b8bdb0125b81968cfcc557dbd',
  'password':"""D[Cvr1bgf9DM^I{C""",
  'container':'CursoSpark',
  'tenantId':'undefined',
  'filename':'iris.csv'
}


In [8]:

from pyspark.sql import SparkSession

# @hidden_cell
# This function is used to setup the access of Spark to your Object Storage. The definition contains your credentials.
# You might want to remove those credentials before you share your notebook.
def set_hadoop_config_with_credentials_f0d6ce325e0f4bc08812229b8d429dbe(name):
    """This function sets the Hadoop configuration so it is possible to
    access data from Bluemix Object Storage using Spark"""

    prefix = 'fs.swift.service.' + name
    hconf = sc._jsc.hadoopConfiguration()
    hconf.set(prefix + '.auth.url', 'https://identity.open.softlayer.com'+'/v3/auth/tokens')
    hconf.set(prefix + '.auth.endpoint.prefix', 'endpoints')
    hconf.set(prefix + '.tenant', '9a0cc60102244d368e96a83f25d4ca89')
    hconf.set(prefix + '.username', '0caf8026c98a4342ac027a05416e6dee')
    hconf.set(prefix + '.password', 'D[Cvr1bgf9DM^I{C')
    hconf.setInt(prefix + '.http.port', 8080)
    hconf.set(prefix + '.region', 'dallas')
    hconf.setBoolean(prefix + '.public', False)

# you can choose any name
name = 'keystone'
set_hadoop_config_with_credentials_f0d6ce325e0f4bc08812229b8d429dbe(name)

# Carregando os dados e gerando um RDD
fileNameOut = 'swift://'+ credentials_1['container'] + '.keystone/iris.csv' 
irisRDD = sc.textFile(fileNameOut)



In [9]:
type(irisRDD)

pyspark.rdd.RDD

In [10]:
from pyspark.sql import Row
from pyspark.ml.feature import StringIndexer
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [11]:
# Colocando o RDD em cache. Esse processo otimiza a performance.
irisRDD.cache()

swift://CursoSpark.keystone/iris.csv MapPartitionsRDD[4] at textFile at NativeMethodAccessorImpl.java:-2

In [12]:
irisRDD.count()

151

In [13]:
irisRDD.take(5)

[u'Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species',
 u'5.1,3.5,1.4,0.2,setosa',
 u'4.9,3,1.4,0.2,setosa',
 u'4.7,3.2,1.3,0.2,setosa',
 u'4.6,3.1,1.5,0.2,setosa']

In [14]:
# Removendo a primeira linha do arquivo (cabeçalho)
irisRDD2 = irisRDD.filter(lambda x: "Sepal" not in x)
irisRDD2.count()

150

## Limpeza dos Dados

In [15]:
# Separando as colunas
irisRDD3 = irisRDD2.map(lambda l: l.split(","))

In [16]:
# Mapeando as colunas
irisRDD4 = irisRDD3.map(lambda p: Row(SEPAL_LENGTH = float(p[0]), SEPAL_WIDTH = float(p[1]), 
                                      PETAL_LENGTH = float(p[2]), PETAL_WIDTH = float(p[3]), 
                                      SPECIES = p[4] ))

In [19]:
# Criando um Dataframe
irisDF = spark.createDataFrame(irisRDD4)
irisDF.cache()

DataFrame[PETAL_LENGTH: double, PETAL_WIDTH: double, SEPAL_LENGTH: double, SEPAL_WIDTH: double, SPECIES: string]

In [20]:
irisDF.take(5)

[Row(PETAL_LENGTH=1.4, PETAL_WIDTH=0.2, SEPAL_LENGTH=5.1, SEPAL_WIDTH=3.5, SPECIES=u'setosa'),
 Row(PETAL_LENGTH=1.4, PETAL_WIDTH=0.2, SEPAL_LENGTH=4.9, SEPAL_WIDTH=3.0, SPECIES=u'setosa'),
 Row(PETAL_LENGTH=1.3, PETAL_WIDTH=0.2, SEPAL_LENGTH=4.7, SEPAL_WIDTH=3.2, SPECIES=u'setosa'),
 Row(PETAL_LENGTH=1.5, PETAL_WIDTH=0.2, SEPAL_LENGTH=4.6, SEPAL_WIDTH=3.1, SPECIES=u'setosa'),
 Row(PETAL_LENGTH=1.4, PETAL_WIDTH=0.2, SEPAL_LENGTH=5.0, SEPAL_WIDTH=3.6, SPECIES=u'setosa')]

In [21]:
# Criando um índice numérico para a coluna de label target
stringIndexer = StringIndexer(inputCol = "SPECIES", outputCol = "IDX_SPECIES")
si_model = stringIndexer.fit(irisDF)
irisNormDF = si_model.transform(irisDF)

In [22]:
irisNormDF.select("SPECIES","IDX_SPECIES").distinct().collect()

[Row(SPECIES=u'versicolor', IDX_SPECIES=0.0),
 Row(SPECIES=u'setosa', IDX_SPECIES=2.0),
 Row(SPECIES=u'virginica', IDX_SPECIES=1.0)]

## Análise Exploratória de Dados

In [23]:
# Estatística descritiva
irisNormDF.describe().show()

+-------+------------------+------------------+------------------+------------------+------------------+
|summary|      PETAL_LENGTH|       PETAL_WIDTH|      SEPAL_LENGTH|       SEPAL_WIDTH|       IDX_SPECIES|
+-------+------------------+------------------+------------------+------------------+------------------+
|  count|               150|               150|               150|               150|               150|
|   mean| 3.758000000000001|1.1993333333333331| 5.843333333333332|3.0573333333333337|               1.0|
| stddev|1.7652982332594662|0.7622376689603467|0.8280661279778634|0.4358662849366978|0.8192319205190404|
|    min|               1.0|               0.1|               4.3|               2.0|               0.0|
|    max|               6.9|               2.5|               7.9|               4.4|               2.0|
+-------+------------------+------------------+------------------+------------------+------------------+



In [24]:
# Correlação entre as variáveis
for i in irisNormDF.columns:
    if not( isinstance(irisNormDF.select(i).take(1)[0][0], str)) :
        print( "Correlação da variável Species com", i, irisNormDF.stat.corr('IDX_SPECIES', i))

('Correla\xc3\xa7\xc3\xa3o da vari\xc3\xa1vel Species com', 'PETAL_LENGTH', -0.649241830764174)
('Correla\xc3\xa7\xc3\xa3o da vari\xc3\xa1vel Species com', 'PETAL_WIDTH', -0.5803770334306263)
('Correla\xc3\xa7\xc3\xa3o da vari\xc3\xa1vel Species com', 'SEPAL_LENGTH', -0.46003915650023686)
('Correla\xc3\xa7\xc3\xa3o da vari\xc3\xa1vel Species com', 'SEPAL_WIDTH', 0.6183715308237433)


IllegalArgumentException: u'requirement failed: Currently correlation calculation for columns with dataType StringType not supported.'

## Pré-Processamento dos Dados

In [25]:
# Criando um LabeledPoint (target, Vector[features])
# Remove colunas não relevantes para o modelo ou com baixa correlação
def transformaVar(row) :
    obj = (row["SPECIES"], row["IDX_SPECIES"], Vectors.dense([row["SEPAL_LENGTH"], row["SEPAL_WIDTH"], 
                                                              row["PETAL_LENGTH"], row["PETAL_WIDTH"]]))
    return obj

In [26]:
irisRDD5 = irisNormDF.rdd.map(transformaVar)

In [27]:
irisRDD5.take(5)

[(u'setosa', 2.0, DenseVector([5.1, 3.5, 1.4, 0.2])),
 (u'setosa', 2.0, DenseVector([4.9, 3.0, 1.4, 0.2])),
 (u'setosa', 2.0, DenseVector([4.7, 3.2, 1.3, 0.2])),
 (u'setosa', 2.0, DenseVector([4.6, 3.1, 1.5, 0.2])),
 (u'setosa', 2.0, DenseVector([5.0, 3.6, 1.4, 0.2]))]

In [29]:
irisDF = spark.createDataFrame(irisRDD5,["species", "label", "features"])
irisDF.select("species","label","features").show(10)
irisDF.cache()

+-------+-----+-----------------+
|species|label|         features|
+-------+-----+-----------------+
| setosa|  2.0|[5.1,3.5,1.4,0.2]|
| setosa|  2.0|[4.9,3.0,1.4,0.2]|
| setosa|  2.0|[4.7,3.2,1.3,0.2]|
| setosa|  2.0|[4.6,3.1,1.5,0.2]|
| setosa|  2.0|[5.0,3.6,1.4,0.2]|
| setosa|  2.0|[5.4,3.9,1.7,0.4]|
| setosa|  2.0|[4.6,3.4,1.4,0.3]|
| setosa|  2.0|[5.0,3.4,1.5,0.2]|
| setosa|  2.0|[4.4,2.9,1.4,0.2]|
| setosa|  2.0|[4.9,3.1,1.5,0.1]|
+-------+-----+-----------------+
only showing top 10 rows



DataFrame[species: string, label: double, features: vector]

## Machine Learning

In [30]:
# Dados de Treino e de Teste
(dados_treino, dados_teste) = irisDF.randomSplit([0.7, 0.3])

In [31]:
dados_treino.count()

106

In [32]:
dados_teste.count()

44

In [33]:
# Construindo o modelo com os dados de treino
dtClassifer = DecisionTreeClassifier(maxDepth = 2, labelCol = "label", featuresCol = "features")
modelo = dtClassifer.fit(dados_treino)

In [34]:
modelo.numNodes
modelo.depth

2

In [35]:
# Previsões com dados de teste
previsoes = modelo.transform(dados_teste)
previsoes.select("prediction","species","label").collect()

[Row(prediction=2.0, species=u'setosa', label=2.0),
 Row(prediction=2.0, species=u'setosa', label=2.0),
 Row(prediction=2.0, species=u'setosa', label=2.0),
 Row(prediction=2.0, species=u'setosa', label=2.0),
 Row(prediction=2.0, species=u'setosa', label=2.0),
 Row(prediction=2.0, species=u'setosa', label=2.0),
 Row(prediction=2.0, species=u'setosa', label=2.0),
 Row(prediction=2.0, species=u'setosa', label=2.0),
 Row(prediction=2.0, species=u'setosa', label=2.0),
 Row(prediction=0.0, species=u'versicolor', label=0.0),
 Row(prediction=0.0, species=u'versicolor', label=0.0),
 Row(prediction=0.0, species=u'versicolor', label=0.0),
 Row(prediction=0.0, species=u'versicolor', label=0.0),
 Row(prediction=0.0, species=u'versicolor', label=0.0),
 Row(prediction=0.0, species=u'versicolor', label=0.0),
 Row(prediction=0.0, species=u'versicolor', label=0.0),
 Row(prediction=0.0, species=u'versicolor', label=0.0),
 Row(prediction=0.0, species=u'versicolor', label=0.0),
 Row(prediction=0.0, species

In [36]:
# Avaliando a acurácia
avaliador = MulticlassClassificationEvaluator(predictionCol = "prediction", labelCol = "label", metricName = "accuracy")
avaliador.evaluate(previsoes)      

0.9772727272727273

In [37]:
# Resumindo as previsões - Confusion Matrix
previsoes.groupBy("label","prediction").count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|   18|
|  2.0|       2.0|    9|
|  1.0|       0.0|    1|
|  0.0|       0.0|   16|
+-----+----------+-----+



# Fim

### Obrigado - Data Science Academy - <a href=http://facebook.com/dsacademy>facebook.com/dsacademybr</a>